<a href="https://colab.research.google.com/github/kepaxabier/proba/blob/main/stanza.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install stanza
import stanza
class Word:
    def __init__(self):
        self.index = None
        self.text = None
        self.lemma = None
        self.upos = None
        self.xpos = None
        self.feats = None
        self.governor = None
        self.dependency_relation = None
class Sentence:
    def __init__(self):
        self.word_list = []
        self.text = None

class Paragraph:
    def __init__(self):
        self.sentence_list = []
        self.text = None
    def sentences_list_to_text(self):
        """
        Metodo que se encarga de obtener la lista de sentencias en modo texto.
        Esto será necesario para el conteo de emoticonos con la herramienta advertools
        """
        list_sentences_str = []
        for s in self.sentence_list:
            list_sentences_str.append(s.text)

        return list_sentences_str
class Document:
    def __init__(self, text, language):
        self.text = text
        self.language = language
        self.paragraph_list = []
        self.words_freq = {}
    def imprimir(self):
        #print(self.text)
        for p in self.paragraph_list:
            sentences_str = p.sentences_list_to_text()
            #print (sentences_str)
            for s in p.sentence_list:
                if not s.text == "":
                    #print(s.text)
                    for w in s.word_list:
                      if w.index and w.text and  w.lemma and w.upos and w.xpos and w.feats and w.governor and w.dependency_relation:
                        print(str(w.index) + "\t" + w.text + "\t" + w.lemma + "\t" + w.upos + "\t" + w.xpos + "\t" + w.feats + "\t" + str(w.governor) + "\t" + str(w.dependency_relation) +"\t")
                      else:
                        print(str(w.index) + "\t" + w.text + "\t" + w.lemma + "\t" + w.upos + "\t" + w.xpos + "\t" + str(w.governor) + "\t" + str(w.dependency_relation) +"\t")
                      
#clase abstracta
class ParserInterface:
  def download_model(self): pass
  def load_model(self): pass
  def get_estructure(self): pass
  def sent2sequence(self, sent):pass
class StanzaInterface(ParserInterface):
    def __init__(self, language, library, directory):
        self.lang = language
        self.lib = library
        self.dir = directory
        self.text = None
        self.textwithparagraphs = None
        self.parser = None #modelo cargado
    def download_model(self): 
      print("-----------You are going to use Stanza library-----------")
      if self.lang == "basque":
        print("-------------You are going to use Basque model-------------")
        MODELS_DIR = self.dir #+ '/eu'
        stanza.download('eu', MODELS_DIR)  # Download the Basque models
      elif self.lang == "english":
        print("-------------You are going to use English model-------------")
        MODELS_DIR = self.dir  # + '/en'
        print("-------------Downloading Stanza English model-------------")
        stanza.download('en', MODELS_DIR)  # Download the English models
      elif self.lang == "spanish":
        print("-------------You are going to use Spanish model-------------")
        MODELS_DIR = self.dir #+ '/es'
        stanza.download('es', MODELS_DIR)  # Download the Spanish models
      elif self.lang == "italian":
        print("-------------You are going to use Italian model-------------")
        MODELS_DIR = self.dir #+ '/it'
        stanza.download('it', MODELS_DIR)  # Download the Italian models
      elif self.lang == "french":
        print("-------------You are going to use French model-------------")
        MODELS_DIR = self.dir #+ '/fr'
        stanza.download('fr', MODELS_DIR)  # Download the French models
      else:
        print("........You cannot use this language...........")
    def load_model(self):
      print("-----------You are going to use Stanza library-----------")
      if self.lang == "basque":
        print("-------------You are going to use Basque model-------------")
        # MODELS_DIR = 'J:\TextSimilarity\eu'
        MODELS_DIR = self.dir # + '/eu'
        config = {'processors': 'tokenize,pos,lemma,depparse',
        # Comma-separated list of processors to use
        'lang': 'eu',  # Language code for the language to build the Pipeline in
        'dir': MODELS_DIR,
        }
        self.parser = stanza.Pipeline(**config)
      elif self.lang == "english":
        print("-------------You are going to use English model-------------")
        MODELS_DIR = self.dir #+ '/en'
        config = {'processors': 'tokenize,mwt,pos,lemma,depparse,ner',  # Comma-separated list of processors to use
                 'lang': 'en',  # Language code for the language to build the Pipeline in
                 'dir': MODELS_DIR,
                }
        self.parser = stanza.Pipeline(**config)
      elif self.lang == "spanish":
        print("-------------You are going to use Spanish model-------------")
        MODELS_DIR = self.dir # + '/es'
        config = {'processors': 'tokenize,mwt,pos,lemma,depparse,ner',
                  # Comma-separated list of processors to use
                  'lang': 'es',  # Language code for the language to build the Pipeline in
                  'dir': MODELS_DIR,
                }
        self.parser = stanza.Pipeline(**config)
      elif self.lang == "italian":
        print("-------------You are going to use Italian model-------------")
        MODELS_DIR = self.dir #+ '/it'
        config = {'processors': 'tokenize,mwt,pos,lemma,depparse',
                  # Comma-separated list of processors to use
                  'lang': 'it',  # Language code for the language to build the Pipeline in
                  'dir': MODELS_DIR,
                }
        self.parser = stanza.Pipeline(**config)
      elif self.lang == "french":
        print("-------------You are going to use Italian model-------------")
        MODELS_DIR = self.dir #+ '/fr'
        config = {'processors': 'tokenize,mwt,pos,lemma,depparse',
                  # Comma-separated list of processors to use
                  'lang': 'fr',  # Language code for the language to build the Pipeline in
                  'dir': MODELS_DIR,
                }
        self.parser = stanza.Pipeline(**config)
    def sent2sequence(self, sent):
        conllword = ""
        for word in sent.words:
            conllword = conllword + " " + str(word.text)
        return conllword
    def get_estructure(self,text): 
      d = Document(text, self.lang)  # ->data = []
      lines = text.split('¬')
      for line in lines:  # paragraph
        if not line.strip() == '':
          p = Paragraph()  # -> paragraph = []
          p.text = line
          doc = self.parser(line)
          for sent in doc.sentences:
            s = Sentence()
            sequence = self.sent2sequence(sent)
            s.text = sequence
            for word in sent.words:
              # Por cada palabra de cada sentencia, creamos un objeto Word que contendra los attrs
              w = Word()
              w.index = str(word.id)  #.index)
              w.text = word.text
              w.lemma = word.lemma
              w.upos = word.upos
              w.xpos = word.xpos
              w.feats = word.feats
              w.governor = word.head #stanza head y stanford governor
              w.dependency_relation = word.deprel
              s.word_list.append(w)
            # print(str(w.index) + "\t" + w.text + "\t" + w.lemma + "\t" + w.upos + "\t" + w.xpos + "\t" + w.feats + "\t" + str(w.governor) + "\t" + str(w.dependency_relation) +"\t")
            p.sentence_list.append(s)  # ->paragraph.append(s)
          d.paragraph_list.append(p)  # ->data.append(paragraph)
      return d  
def main():
  # Carga del modelo Stanford/NLPCube
  language="english"
  model="stanza"
  directory="."
  text="I am professor."
  cargador = StanzaInterface(language, model, directory)
  cargador.download_model()
  cargador.load_model()
  document = cargador.get_estructure(text)
  document.imprimir()
if __name__ == "__main__":
   main()                      

-----------You are going to use Stanza library-----------
-------------You are going to use English model-------------
-------------Downloading Stanza English model-------------


2022-01-10 10:43:44 INFO: Downloading default packages for language: en (English)...


2022-01-10 10:43:59 INFO: Finished downloading models and saved to ..
2022-01-10 10:43:59 WARNING: Can not find mwt: default from official model list. Ignoring it.
2022-01-10 10:43:59 INFO: Loading these models for language: en (English):
| Processor | Package   |
-------------------------
| tokenize  | combined  |
| pos       | combined  |
| lemma     | combined  |
| depparse  | combined  |
| ner       | ontonotes |

2022-01-10 10:43:59 INFO: Use device: cpu
2022-01-10 10:43:59 INFO: Loading: tokenize


-----------You are going to use Stanza library-----------
-------------You are going to use English model-------------


2022-01-10 10:44:00 INFO: Loading: pos
2022-01-10 10:44:00 INFO: Loading: lemma
2022-01-10 10:44:00 INFO: Loading: depparse
2022-01-10 10:44:00 INFO: Loading: ner
2022-01-10 10:44:01 INFO: Done loading processors!


1	I	I	PRON	PRP	Case=Nom|Number=Sing|Person=1|PronType=Prs	3	nsubj	
2	am	be	AUX	VBP	Mood=Ind|Number=Sing|Person=1|Tense=Pres|VerbForm=Fin	3	cop	
3	professor	professor	NOUN	NN	0	root	
4	.	.	PUNCT	.	3	punct	
